In [15]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from time import time
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.applications.inception_v3 import InceptionV3
import tensorflow as tf
import glob
import math
from collections import Counter
from matplotlib import pyplot as plt
import numpy as np
import cv2
import glob
import imutils
import os
import pandas as pd
from tqdm import tqdm
from urllib.request import urlretrieve
from os import listdir
from os.path import isfile, join, exists
from zipfile import ZipFile
from scipy.cluster.vq import *

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

from skimage import exposure
from skimage import feature
from skimage.feature import daisy,hog
from skimage import io
from skimage.color import rgb2gray
import skimage

%matplotlib inline

#hide warnings
import warnings
warnings.filterwarnings('ignore')

# 5. Prepare dataset

In [16]:
IMG_SIZE = (299, 299, 3)
BATCH_SIZE = 16

In [17]:
DATASET_DIRECTORY = "MantaRay"

In [18]:
# Map each class to an integer label
class_mapping = {}
index = 0
class_images = {}
# Create dictionary to map integer labels to individuals
# Class_images will record number of images for each class
for directory in glob.glob(DATASET_DIRECTORY + '/*'):
    class_mapping[index] = directory.split("/")[1]
    class_images[index] = len(glob.glob(directory + '/*_CR.jpg')) + len(glob.glob(directory + '/*_CR.JPG'))
    index += 1

In [19]:
image_arrays = []
image_labels = []
image_paths = []

for label, individual in class_mapping.items():
    for directory in glob.glob(DATASET_DIRECTORY + '/*'):
        if directory.split("/")[1] == individual:
            image_directory = directory
            break
 
    for image in glob.glob(directory + '/*_EH.jpg'):
        image_paths.append(image)
        image = plt.imread(image)
        image_arrays.append(image)
        image_labels.append(label)
        
    for image in glob.glob(directory + '/*_EH.JPG'):
        image_paths.append(image)
        image = plt.imread(image)
        image_arrays.append(image)
        image_labels.append(label)
        
image_arrays = np.array(image_arrays)
image_labels = np.array(image_labels)
print(image_arrays.shape, image_labels.shape)

(720,) (720,)


# 6. Data augmentation for SIFT

In [656]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=0.2,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

for n in range(len(image_paths)):
    img = load_img(image_paths[n])  # this is a PIL image, please replace to your own file path
    x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory

    i = 0
    for batch in datagen.flow(x, 
                              batch_size=1,
                              save_to_dir='MantaRay/' + image_paths[n].split('/')[1],  
                              save_prefix= 'AUG_' + image_paths[n].split('/')[2], 
                              save_format='jpg'):
        i += 1
        if i > 2:
            break  # otherwise the generator would loop indefinitely


# 7. SURF & SIFT extraction

In [40]:
image_arrays = []
image_labels = []
image_paths = []
#root_image_directory = "images/faces/"
for label, individual in class_mapping.items():
    for directory in glob.glob(DATASET_DIRECTORY + '/*'):
        if directory.split("/")[1] == individual:
            image_directory = directory
            break
 
    for image in glob.glob(directory + '/*_EH.jpg'):
        image_paths.append(image)
        image = plt.imread(image)
        image_arrays.append(image)
        image_labels.append(label)
        
    for image in glob.glob(directory + '/*_EH.JPG'):
        image_paths.append(image)
        image = plt.imread(image)
        image_arrays.append(image)
        image_labels.append(label)
        
image_arrays = np.array(image_arrays)
image_labels = np.array(image_labels)
print(image_arrays.shape, image_labels.shape)

(720,) (720,)


In [33]:
aug_image_arrays = []
aug_image_labels = []
aug_image_paths = []
#root_image_directory = "images/faces/"
for label, individual in class_mapping.items():
    for directory in glob.glob(DATASET_DIRECTORY + '/*'):
        if directory.split("/")[1] == individual:
            image_directory = directory
            break
 
    for image in glob.glob(directory + '/AUG_*.jpg'):
        aug_image_paths.append(image)
        image = plt.imread(image)
        aug_image_arrays.append(image)
        aug_image_labels.append(label)

aug_image_arrays = np.array(aug_image_arrays)
aug_image_labels = np.array(aug_image_labels)
print(aug_image_arrays.shape, aug_image_labels.shape)

(2160,) (2160,)


In [657]:
def pickle_keypoints(keypoints, descriptors):
    i = 0
    temp_array = []
    for point in keypoints:
        temp = (point.pt, point.size, point.angle, point.response, point.octave,
        point.class_id, descriptors[i])     
        ++i
        temp_array.append(temp)
    return temp_array

def unpickle_keypoints(array):
    keypoints = []
    descriptors = []
    for point in array:
        temp_feature = cv2.KeyPoint(x=point[0][0],y=point[0][1],_size=point[1], _angle=point[2], _response=point[3], _octave=point[4], _class_id=point[5])
        temp_descriptor = point[6]
        keypoints.append(temp_feature)
        descriptors.append(temp_descriptor)
    return keypoints, np.array(descriptors)

In [671]:
# import cPickle as pickle

for i in range(len(image_paths)):
    # Load the images
    img1 =cv2.imread(image_paths[i])

    # Convert them to greyscale
    grey_img1 =cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

    # SURF extraction
    surf = cv2.xfeatures2d.SURF_create()
    kp1, desc1 = surf.detectAndCompute(grey_img1,None)

    #Store and Retrieve keypoint features
    temp_array = []
    temp = pickle_keypoints(kp1, desc1)
    temp_array.append(temp)

    pickle.dump(temp_array, open('MantaRay/'+image_paths[i].split('/')[1]+'/'+image_paths[i].split('/')[2].split('_')[0]+"_SURF.p", "wb"))

In [675]:
for i in range(len(image_paths)):
    # Load the images
    img1 =cv2.imread(image_paths[i])

    # Convert them to greyscale
    grey_img1 =cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

    # SIFT extraction
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(grey_img1,None)

    #Store and Retrieve keypoint features
    temp_array = []
    temp = pickle_keypoints(kp1, desc1)
    temp_array.append(temp)

    pickle.dump(temp_array, open('MantaRay/'+image_paths[i].split('/')[1]+'/'+image_paths[i].split('/')[2].split('_')[0]+"_SIFT.p", "wb"))

In [680]:
for i in range(len(aug_image_paths)):
    # Load the images
    img1 =cv2.imread(aug_image_paths[i])

    # Convert them to greyscale
    grey_img1 =cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

    # SURF extraction
    surf = cv2.xfeatures2d.SURF_create()
    kp1, desc1 = surf.detectAndCompute(grey_img1,None)

    #Store and Retrieve keypoint features
    temp_array = []
    temp = pickle_keypoints(kp1, desc1)
    temp_array.append(temp)

    pickle.dump(temp_array, open('MantaRay/'+aug_image_paths[i].split('/')[1]+'/'+aug_image_paths[i].split('/')[2]+"_SURF.p", "wb"))

In [681]:
for i in range(len(aug_image_paths)):
    # Load the images
    img1 =cv2.imread(aug_image_paths[i])

    # Convert them to greyscale
    grey_img1 =cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

    # SURF extraction
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, desc1 = sift.detectAndCompute(grey_img1,None)

    #Store and Retrieve keypoint features
    temp_array = []
    temp = pickle_keypoints(kp1, desc1)
    temp_array.append(temp)

    pickle.dump(temp_array, open('MantaRay/'+aug_image_paths[i].split('/')[1]+'/'+aug_image_paths[i].split('/')[2]+"_SIFT.p", "wb"))

In [672]:
#Retrieve Keypoint Features
keypoints_database = pickle.load(open("MantaRay/Four/Four 3_SURF.p", "rb"))
kp1, desc1 = unpickle_keypoints(keypoints_database[0])

# 8. SIFT Matcher with 2160 aug, 720 aug, and no aug

In [35]:
total_paths = image_paths + aug_image_paths

In [36]:
len(total_paths)

2880

In [685]:
total_labels = []
for i in range(len(total_paths)):
    total_labels.append(total_paths[i].split('/')[1])

In [728]:
matching_score = []

for i in range(len(total_paths)-1):
    print(i)
    scores = []
    img1 = cv2.imread(total_paths[i],0)     # queryImage
    sift = cv2.xfeatures2d.SIFT_create()

    kp1, des1 = sift.detectAndCompute(img1,None)
    for j in range(i+1,len(total_paths)):
        
        img2 = cv2.imread(total_paths[j],0) # trainImage

        kp2, des2 = sift.detectAndCompute(img2,None)

        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)

        good = []
        for m,n in matches:
            if m.distance < 0.75*n.distance:
                good.append([m])

        scores.append(len(good)/np.min([len(des1),len(des2)]))
        
    matching_score.append(scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190


KeyboardInterrupt: 

In [37]:
with open('full_aug_testset.txt', 'w') as f:
    for item in total_paths[:190]:
        f.write("%s\n" % item)

In [729]:
submatching_score = []

for i in range(1439):
    print(i)
    scores = []
    img1 = cv2.imread(total_paths[i],0)     # queryImage
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    for j in range(i+1,1440):
        
        img2 = cv2.imread(total_paths[j],0) # trainImage

        # find the keypoints and descriptors with SIFT
        kp2, des2 = sift.detectAndCompute(img2,None)
        
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        
        # Apply ratio test
        good = []
        for m,n in matches:
            if m.distance < 0.75*n.distance:
                good.append([m])

        scores.append(len(good)/np.min([len(des1),len(des2)]))
        
    submatching_score.append(scores)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144


KeyboardInterrupt: 

In [768]:
for i in range(144,1439):
    print(i)
    scores = []
    img1 = cv2.imread(total_paths[i],0)     # queryImage
    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    for j in range(i+1,1440):
        
        img2 = cv2.imread(total_paths[j],0) # trainImage

        # find the keypoints and descriptors with SIFT
        kp2, des2 = sift.detectAndCompute(img2,None)
        
        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        
        # Apply ratio test
        good = []
        for m,n in matches:
            if m.distance < 0.75*n.distance:
                good.append([m])

        scores.append(len(good)/np.min([len(des1),len(des2)]))
        
    submatching_score.append(scores)

144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208


KeyboardInterrupt: 

In [38]:
with open('half_aug_testset.txt', 'w') as f:
    for item in total_paths[:208]:
        f.write("%s\n" % item)

In [823]:
score_list = []
for i in range(len(matching_score)):
    score_tmp = matching_score[i]
    score_tmp.insert(0,1) # matching with the individual itself
    for j in reversed(range(i)):
        score_tmp.insert(0,matching_score[j][i-j])
    score_list.append(score_tmp)

In [758]:
# Calculating top-k accuracies for full matching (2880) images
total_1 = 0
total_5 = 0
total_10 = 0
total_25 = 0
total_50 = 0
total_75 = 0
total_100 = 0
true_1 = 0
true_5 = 0
true_10 = 0
true_25 = 0
true_50 = 0
true_75 = 0
true_100 = 0

for i in range(len(score_list)):
        true_label = total_labels[i]
        top_1_idx = np.argsort(score_list[i])[-1:]
        top_5_idx = np.argsort(score_list[i])[-5:]
        top_10_idx = np.argsort(score_list[i])[-10:]
        top_25_idx = np.argsort(score_list[i])[-25:]
        top_50_idx = np.argsort(score_list[i])[-50:]
        top_75_idx = np.argsort(score_list[i])[-75:]
        top_100_idx = np.argsort(score_list[i])[-100:]
        total_1 += 1
        total_5 += 1
        total_10 += 1
        total_25 += 1
        total_50 += 1
        total_75 += 1
        total_100 += 1
        for index in top_1_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    #print(i)
                    true_1 += 1
                    break
        for index in top_5_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_5 += 1
                    break
        for index in top_10_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_10 += 1
                    break
        for index in top_25_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_25 += 1
                    break
        for index in top_50_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_50 += 1
                    break
        for index in top_75_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_75 += 1
                    break
        for index in top_100_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_100 += 1
                    break

In [759]:
print(true_1/total_1)
print(true_5/total_5)
print(true_10/total_10)
print(true_25/total_25)
print(true_50/total_50)
print(true_75/total_75)
print(true_100/total_100)

0.18421052631578946
0.6684210526315789
0.8842105263157894
0.9842105263157894
1.0
1.0
1.0


In [770]:
score_list_2 = []
for i in range(len(submatching_score)):
    score_tmp = submatching_score[i]
    score_tmp.insert(0,1)
    for j in reversed(range(i)):
        score_tmp.insert(0,submatching_score[j][i-j])
    score_list_2.append(score_tmp)

In [771]:
# Calculating top-k accuracies for half matching (1440) images
total_1 = 0
total_5 = 0
total_10 = 0
total_25 = 0
total_50 = 0
total_75 = 0
total_100 = 0
true_1 = 0
true_5 = 0
true_10 = 0
true_25 = 0
true_50 = 0
true_75 = 0
true_100 = 0

for i in range(len(score_list_2)):
        true_label = total_labels[i]
        top_1_idx = np.argsort(score_list_2[i])[-1:]
        top_5_idx = np.argsort(score_list_2[i])[-5:]
        top_10_idx = np.argsort(score_list_2[i])[-10:]
        top_25_idx = np.argsort(score_list_2[i])[-25:]
        top_50_idx = np.argsort(score_list_2[i])[-50:]
        top_75_idx = np.argsort(score_list_2[i])[-75:]
        top_100_idx = np.argsort(score_list_2[i])[-100:]
        total_1 += 1
        total_5 += 1
        total_10 += 1
        total_25 += 1
        total_50 += 1
        total_75 += 1
        total_100 += 1
        for index in top_1_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    #print(i)
                    true_1 += 1
                    break
        for index in top_5_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_5 += 1
                    break
        for index in top_10_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_10 += 1
                    break
        for index in top_25_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_25 += 1
                    break
        for index in top_50_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_50 += 1
                    break
        for index in top_75_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_75 += 1
                    break
        for index in top_100_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_100 += 1
                    break

In [772]:
print(true_1/total_1)
print(true_5/total_5)
print(true_10/total_10)
print(true_25/total_25)
print(true_50/total_50)
print(true_75/total_75)
print(true_100/total_100)

0.08173076923076923
0.5336538461538461
0.8894230769230769
0.9951923076923077
1.0
1.0
1.0


In [831]:
import timeit

orimatching_score = []
time = []
for i in range(719):
    print(i)
    start = timeit.timeit()
    scores = []
    img1 = cv2.imread(total_paths[i],0)
    sift = cv2.xfeatures2d.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1,None)
    for j in range(i+1,720):
        
        img2 = cv2.imread(total_paths[j],0)
        kp2, des2 = sift.detectAndCompute(img2,None)
        
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary
        flann = cv2.FlannBasedMatcher(index_params,search_params)
        matches = flann.knnMatch(des1,des2,k=2)
        
        good = []
        for m,n in matches:
            if m.distance < 0.75*n.distance:
                good.append([m])

        scores.append(len(good)/np.min([len(des1),len(des2)]))
        
    orimatching_score.append(scores)
    end = timeit.timeit()
    time.append(end - start)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [834]:
score_list_4 = []
for i in range(len(orimatching_score)):
    score_tmp = orimatching_score[i]
    score_tmp.insert(0,1)
    for j in reversed(range(i)):
        score_tmp.insert(0,orimatching_score[j][i-j])
    score_list_4.append(score_tmp)

In [835]:
# Calculating top-k accuracies for original matching (720) images
total_1 = 0
total_5 = 0
total_10 = 0
total_25 = 0
total_50 = 0
total_75 = 0
total_100 = 0
true_1 = 0
true_5 = 0
true_10 = 0
true_25 = 0
true_50 = 0
true_75 = 0
true_100 = 0

for i in range(len(score_list_4)):
        true_label = total_labels[i]
        top_1_idx = np.argsort(score_list_4[i])[-1:]
        top_5_idx = np.argsort(score_list_4[i])[-5:]
        top_10_idx = np.argsort(score_list_4[i])[-10:]
        top_25_idx = np.argsort(score_list_4[i])[-25:]
        top_50_idx = np.argsort(score_list_4[i])[-50:]
        top_75_idx = np.argsort(score_list_4[i])[-75:]
        top_100_idx = np.argsort(score_list_4[i])[-100:]
        total_1 += 1
        total_5 += 1
        total_10 += 1
        total_25 += 1
        total_50 += 1
        total_75 += 1
        total_100 += 1
        for index in top_1_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    #print(i)
                    true_1 += 1
                    break
        for index in top_5_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_5 += 1
                    break
        for index in top_10_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_10 += 1
                    break
        for index in top_25_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_25 += 1
                    break
        for index in top_50_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_50 += 1
                    break
        for index in top_75_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_75 += 1
                    break
        for index in top_100_idx:
            #if index < len(aug_image_labels):
                if total_labels[index] == true_label:
                    true_100 += 1
                    break

In [836]:
print(true_1/total_1)
print(true_5/total_5)
print(true_10/total_10)
print(true_25/total_25)
print(true_50/total_50)
print(true_75/total_75)
print(true_100/total_100)

0.37337662337662336
0.9642857142857143
0.9967532467532467
1.0
1.0
1.0
1.0


In [39]:
with open('no_aug_testset.txt', 'w') as f:
    for item in total_paths[:308]:
        f.write("%s\n" % item)